# Get Artwork Images
Pull the Artwork image files from the website.

In [1]:
import pandas as pd
import requests
import sqlite3
import time
import sys

In [ ]:
data_file_path = "./data/"

## Connect to the Database

In [3]:
# Create a DB connection between python and the file system
conn = sqlite3.connect(''.join([data_file_path,"/database/artist.db"]))

## Extract Image Name & URL From the Database
__NOTE:__ the predefined _image_filename_ contains an _image_tag_ value. This identifies both the Artist and the Artwork.

In [4]:
# Simple SELECT (1 table)
image_data = pd.read_sql_query("""SELECT IMAGE_FILENAME,
                                         URL
                                  FROM   ARTWORK_IMAGE;""", conn)

## Request the Image Files
The images are pulled and saved to disk with the predefined filename. <br/>
__NOTE:__ As each file is downloaded the _Downloaded_ count below this code cell is incremeted by one. <br/>
__NOTE:__ A short pause was introduced every 50 images so that the webserver is not bombarded with a list of requests.

In [5]:
# set download counter
download_count = 0

# loop throught the dataset containing image names and urls fro the database select
for index, row in image_data.iterrows():

    try:
        # make a GET request for the image file
        response = requests.get(row["url"])

        # open a file on the fs and write the response data from the GET
        with open("".join([data_file_path,"images/", row["image_filename"]]), "wb") as image_file:
            image_file.write(response.content)
    except:
        # heads up should an image fail
        print("".join(["ERROR RETREIVING: [", row["image_filename"] , "] URL: ", row["url"]]))
    
    # increment the download counter
    download_count += 1
    
    sys.stdout.write(f"\rDownloaded: {str(download_count).zfill(4)}")
    
    # at the point of 50 images being downloaded pause for 5 seconds (don't look like a DDOS)
    if download_count % 50 == 0:
        time.sleep(5)

Downloaded: 3306